<a href="https://colab.research.google.com/github/MohammadNyundo/TRADING_AUTOMATION/blob/master/Fantasy_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Import Modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

import statistics

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from google.colab import drive


# xgboost popular gradient boosting library for increase perfomance and speed
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras import layers
from tensorflow.keras import losses
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize

from textblob import TextBlob

nltk.download('punkt')  # Download the punkt tokenizer data


# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")


Ran the import statements.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#@title Print the version of tf for compatibility
print(tf.__version__)

2.18.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#@title Exploring the dataset
df = pd.read_csv('/content/drive/MyDrive/Datasets/generated dataset.csv')
print(df.head())
print(df.tail())

# 1 for AI generated and 0 for human generated


                                                Text  Label
0  Dear Florida State Senator, Although many coul...      0
1  Thousands of people around the world use cars....      0
2  Although numerous amounts of people want to ba...      0
3  There are many different things that you could...      0
4  Dear Senator, The Electoral College as controv...      0
                                                  Text  Label
995    Pioneering Sustainable Urban Living  In an e...      1
996    The Path to Sustainable Urban Living  In an ...      1
997    A Paradigm Shift in Urban Living  In an era ...      1
998    Revolutionizing Urban Living  In an age defi...      1
999    Pioneering Sustainable Urban Living  In an a...      1


In [6]:
#@title Basic info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    1000 non-null   object
 1   Label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [7]:
#@title check for null values
print(df.isnull().sum())

# no null values

Text     0
Label    0
dtype: int64


In [8]:
#@ creating a copy of the dataset
df_copy = df.copy()
df_copy.head()


,Text,Label
0,"Dear Florida State Senator, Although many coul...",0
1,Thousands of people around the world use cars....,0
2,Although numerous amounts of people want to ba...,0
3,There are many different things that you could...,0
4,"Dear Senator, The Electoral College as controv...",0


In [9]:
#@title Balancing the dataset
df_label_human = df_copy[df_copy['Label'] == 0]
df_label_AI = df_copy[df_copy['Label'] == 1]

print(len(df_label_human))
print(len(df_label_AI))

# our dataset is balanced with 500 human examples and 500 AI examples

500
500


In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
#@title converting the column 'Text' and 'Label' of D.F to lists

text = df_copy.iloc[100]['Text']
texts = df_copy['Text'].tolist()
labels = df_copy['Label'].tolist()

# processing the text
# Preprocess the input text
def preprocess_text(text):
    # Tokenize the text
    tokens = re.split(r'\W+', text.lower())
    # Remove stopwords
    tokens = [token for token in tokens if token not in ENGLISH_STOP_WORDS]
    # Rejoin the tokens into a single string
    processed_text = ' '.join(tokens)
    return processed_text

text1 = preprocess_text(text)


#printing to inspect the data
print(text1, '\n')
print(texts[-110], '\n')
print(labels[15], '\n')
print(len(texts), '\t', len(labels))

day people wake ready work school way vary people drive cars s change helping environment relaxed advantages limiting car usage reducing car usage emmitions produced automobiles reduced help environment cars release gas air causes smog traps gases inhibits able escape atmosphere largely congested cities smog harmful 147 micrograms particulate matter seen paris france paragraph 17 paris enforced partial driving ban smog just day reduced driving smog clears didn t enforce ban day paragraph 19 passenger cars responsible 12 percent greenhouse gas emissions europe 50 percent united states paragraph 5 staggering statistic open eyes cars really effect environment just week didn t use car reduce greenhouse gases drastically united states today recent studies shown americans buying fewer cars driving paragraph 29 1995 number miles driven person dropped 9 percent paragraph 32 lesser people buying driving new cars gas emissions change drastically years help evrioment exonentialy drive car places 

In [11]:
#@title Number of Sentences
def count_sentences_total(text):
    paragraphs = text.split('\n\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    total_sentence_count = 0

    for paragraph in paragraphs:
        sentences = re.split(r'[.?!+]', paragraph)
        sentences = [s for s in sentences if s.strip()]
        total_sentence_count += len(sentences)

    return total_sentence_count

total_sentence_count = count_sentences_total(text1)
print(total_sentence_count)

1


In [12]:
#@title Counting number of words
def count_total_words(text):
    paragraphs = text.split('\n\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    total_word_count = 0

    for paragraph in paragraphs:
        sentences = re.split(r'[.?!+]', paragraph)
        for sentence in sentences:
            words = sentence.split()
            total_word_count += len(words)

    return total_word_count

total_word_count = count_total_words(text1)
print(total_word_count)



277


In [13]:
#@title Total number of parantheses
def detect_total_parantheses(text):
    paragraphs = text.split('\n\n')

    total_parantheses_presence = 0

    for paragraph in paragraphs:
        if '(' in paragraph or ')' in paragraph:
            total_parantheses_presence += 1

    return total_parantheses_presence

total_parantheses_presence = detect_total_parantheses(text1)
print(total_parantheses_presence)


0


In [14]:
#@title Number of dashes
def detect_total_dash(text):
    paragraphs = text.split('\n\n')

    total_dash_presence = 0

    for paragraph in paragraphs:
        if '-' in paragraph:
            total_dash_presence += 1

    return total_dash_presence

total_dash_presence = detect_total_dash(text1)
print(total_dash_presence)

0


In [15]:
#@title Total number of colons and semicolons
def detect_total_semicolon_colon(text):
    paragraphs = text.split('\n\n')

    total_punctuation_presence = 0

    for paragraph in paragraphs:
        if ';' in paragraph or ':' in paragraph:
            total_punctuation_presence += 1

    return total_punctuation_presence

total_semicolon_colon_presence = detect_total_semicolon_colon(text1)
print(total_semicolon_colon_presence)

0


In [16]:
#@title Total Number of question marks
def detect_total_question_mark(text):
    paragraphs = text.split('\n\n')

    total_question_mark_presence = 0

    for paragraph in paragraphs:
        if '?' in paragraph:
            total_question_mark_presence += 1

    return total_question_mark_presence

total_question_mark_presence = detect_total_question_mark(text1)
print(total_question_mark_presence)

0


In [17]:
#@title Total number of Apostrophes
def detect_total_apostrophe(text):
    paragraphs = text.split('\n\n')

    total_apostrophe_presence = 0

    for paragraph in paragraphs:
        if "'" in paragraph:
            total_apostrophe_presence += 1

    return total_apostrophe_presence

total_apostrophe_presence = detect_total_apostrophe(text1)
print(total_apostrophe_presence)

0


In [18]:
#@title Calculating the standard deviation of sentence lenghts entire text
def total_std_dev_sentence_length(text):
    paragraphs = text.split('\n\n')

    all_sentence_lengths = []

    for paragraph in paragraphs:
        sentences = re.split(r'[.?!]+', paragraph)
        sentence_lengths = [len(sentence.split()) for sentence in sentences if sentence.strip()]
        all_sentence_lengths.extend(sentence_lengths)

    if len(all_sentence_lengths) > 1:
        total_std_dev = statistics.stdev(all_sentence_lengths)
        total_std_dev = round(total_std_dev, 2)
    else:
        total_std_dev = 0  # if not enough sentences

    return total_std_dev

total_std_dev = total_std_dev_sentence_length(text1)
print(total_std_dev)

0


In [19]:
#@title Calculating the difference in mean sentence lenght per consequtive paragraphs
def total_mean_diff_sentence_length(text):
    paragraphs = text.split('\n\n')

    all_sentence_lengths = []

    for paragraph in paragraphs:
        sentences = re.split(r'[.?!]+', paragraph)
        sentence_lengths = [len(sentence.split()) for sentence in sentences if sentence.strip()]
        all_sentence_lengths.extend(sentence_lengths)

    differences = [abs(all_sentence_lengths[i] - all_sentence_lengths[i+1]) for i in range(len(all_sentence_lengths) - 1)]

    if differences:
        total_mean_diff = statistics.mean(differences)
        total_mean_diff = round(total_mean_diff, 2)
    else:
        total_mean_diff = 0  # handle where no consecutive sentences or only empty sentences are present.

    return total_mean_diff

total_mean_diff = total_mean_diff_sentence_length(text1)
print(total_mean_diff)


0


In [20]:
#@title detecting presence of short Sentences
def detect_total_short_sentences(text):
    paragraphs = text.split('\n\n')

    has_short_sentence = any(any(len(sentence.split()) < 11 for sentence in re.split(r'[.?!]+', paragraph) if sentence.strip()) for paragraph in paragraphs)

    return 1 if has_short_sentence else 0

total_short_sentence_presence = detect_total_short_sentences(text1)
print(total_short_sentence_presence)

0


In [21]:
#@title Detecting long sentences in the paragraph
def detect_total_long_sentences(text):
    paragraphs = text.split('\n\n')

    has_long_sentence = any(any(len(sentence.split()) > 34 for sentence in re.split(r'[.?!]+', paragraph) if sentence.strip()) for paragraph in paragraphs)

    return 1 if has_long_sentence else 0

total_long_sentence_presence = detect_total_long_sentences(text1)
print(total_long_sentence_presence)

1


In [22]:
#@title Checking if there is a number in the entire text
def detect_total_numbers(text):
    paragraphs = text.split('\n\n')

    has_number = any(any(char.isdigit() for char in paragraph) for paragraph in paragraphs)

    return 1 if has_number else 0

total_number_presence = detect_total_numbers(text1)
print(total_number_presence)

1


In [23]:
#@title Checks the ratio of capital letters to periods
def detect_total_capital_vs_periods(text):
    paragraphs = text.split('\n\n')

    has_condition = any(sum(1 for char in paragraph if char.isupper()) >= 2 * paragraph.count('.') for paragraph in paragraphs)

    return 1 if has_condition else 0

total_capital_vs_period_presence = detect_total_capital_vs_periods(text1)
print(total_capital_vs_period_presence)

1


In [24]:
#@title Performimg sentence tokenization
def tokenize_text(text):
    paragraphs = text.split('\n\n')
    paragraphs = [p.strip() for p in paragraphs if p.strip()]

    # Tokenize words in each sentence of each paragraph
    tokenized_text = []
    for paragraph in paragraphs:
        sentences = re.split(r'[.?!+]', paragraph)
        sentences = [s.strip() for s in sentences if s.strip()]

        tokenized_sentences = []
        for sentence in sentences:
            words = sentence.split()
            words = [w.strip() for w in words if w.strip()]
            tokenized_sentences.append(words)

        tokenized_text.append(tokenized_sentences)

    return tokenized_text

tokenized_text = tokenize_text(text1)
print(tokenized_text)

[[['day', 'people', 'wake', 'ready', 'work', 'school', 'way', 'vary', 'people', 'drive', 'cars', 's', 'change', 'helping', 'environment', 'relaxed', 'advantages', 'limiting', 'car', 'usage', 'reducing', 'car', 'usage', 'emmitions', 'produced', 'automobiles', 'reduced', 'help', 'environment', 'cars', 'release', 'gas', 'air', 'causes', 'smog', 'traps', 'gases', 'inhibits', 'able', 'escape', 'atmosphere', 'largely', 'congested', 'cities', 'smog', 'harmful', '147', 'micrograms', 'particulate', 'matter', 'seen', 'paris', 'france', 'paragraph', '17', 'paris', 'enforced', 'partial', 'driving', 'ban', 'smog', 'just', 'day', 'reduced', 'driving', 'smog', 'clears', 'didn', 't', 'enforce', 'ban', 'day', 'paragraph', '19', 'passenger', 'cars', 'responsible', '12', 'percent', 'greenhouse', 'gas', 'emissions', 'europe', '50', 'percent', 'united', 'states', 'paragraph', '5', 'staggering', 'statistic', 'open', 'eyes', 'cars', 'really', 'effect', 'environment', 'just', 'week', 'didn', 't', 'use', 'car'

In [25]:
#@ title Analyzing the sentiment of the entire text
from nltk.sentiment import SentimentIntensityAnalyzer

# Download VADER lexicon.
nltk.download('vader_lexicon')


def analyze_sentiment(text):
    # Initialize VADER sentiment analyzer
    sia = SentimentIntensityAnalyzer()

    # Calculate compound sentiment score for the entire text
    sentiment_score = sia.polarity_scores(text)['compound']

    # Classify sentiment based on the compound score
    if sentiment_score >= 0.05:
        sentiment = 1 # for positive
    elif sentiment_score <= -0.05:
        sentiment = -1 # for negative
    else:
        sentiment = 0

    return sentiment

sentiment = analyze_sentiment(text1)
print(sentiment)

1


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [26]:
def detect_repetitive_language(text):
    tokens = nltk.word_tokenize(text)
    freq_dist = nltk.FreqDist(tokens)

    # Calculate the frequency of the most common words
    most_common_words = sorted(freq_dist, key=freq_dist.get, reverse=True)[:10]
    num_common_words = sum(freq_dist[word] > 5 for word in most_common_words)

    if num_common_words > 5:
        return 1
    else:
        return 0

repetitive_language = detect_repetitive_language(text1)
print(repetitive_language)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
#@title Checking for specific words
def detect_word_although_total(text):
    paragraphs = text.split('\n\n')
    has_word_although = any('although' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_although else 0

def detect_word_however_total(text):
    paragraphs = text.split('\n\n')
    has_word_however = any('however' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_however else 0

def detect_word_but_total(text):
    paragraphs = text.split('\n\n')
    has_word_but = any('but' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_but else 0

def detect_word_because_total(text):
    paragraphs = text.split('\n\n')
    has_word_because = any('because' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_because else 0

def detect_word_this_total(text):
    paragraphs = text.split('\n\n')
    has_word_this = any('this' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_this else 0

def detect_word_others_researchers_total(text):
    paragraphs = text.split('\n\n')
    has_word_others_researchers = any('others' in paragraph.lower() or 'researchers' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_others_researchers else 0

def detect_word_etc_total(text):
    paragraphs = text.split('\n\n')
    has_word_etc = any('etc.' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_etc else 0

def detect_word_in_conclusion_total(text):
    paragraphs = text.split('\n\n')
    has_word_in_conclusion = any('conclusion' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_in_conclusion else 0

In [ ]:
# Preprocess the input text
def preprocess_text(text):
    # Tokenize the text
    tokens = re.split(r'\W+', text.lower())
    # Remove stopwords
    tokens = [token for token in tokens if token not in ENGLISH_STOP_WORDS]
    # Rejoin the tokens into a single string
    processed_text = ' '.join(tokens)
    return processed_text

preprocessed_texts = []
for text in texts:
    preprocessed_text = preprocess_text(text)
    preprocessed_texts.append(preprocessed_text)

# Loop through the preprocessed texts
results = []
for text in tqdm(preprocessed_texts):
    result = {
        'total_number_sentences': count_sentences_total(text),
        'total_number_words': count_total_words(text),
        'total_number_parantheses': detect_total_parantheses(text),
        'total_number_dash': detect_total_dash(text),
        'total_colons': detect_total_semicolon_colon(text),
        'total_number_questionmark': detect_total_question_mark(text),
        'total_number_apostrophe': detect_total_apostrophe(text),
        'std_dev_sentence_length': total_std_dev_sentence_length(text),
        'mean_diff_consecutive_sentence': total_mean_diff_sentence_length(text),
        'short_sentence_presence': detect_total_short_sentences(text),
        'long_sentence_presence': detect_total_long_sentences(text),
        'numbers_presence': detect_total_numbers(text),
        'capital_vs_period_presence': detect_total_capital_vs_periods(text),
        'word_although_presence': detect_word_although_total(text),
        'word_however_presence': detect_word_however_total(text),
        'word_but_presence': detect_word_but_total(text),
        'word_this_presence': detect_word_this_total(text),
        'word_others_researchers_presence': detect_word_others_researchers_total(text),
        'word_et_presence': detect_word_etc_total(text),
        'word_inconclusion_presence': detect_word_in_conclusion_total(text),
        'repetitive_words': detect_repetitive_language(text),
        'Sentiment': analyze_sentiment(text),
        # 'label': df_copy['Label']
    }
    results.append(result)

In [ ]:
#@title Converting the results to a dataFrame to use as feature
dataFrame_features = pd.DataFrame(results)
print(len(dataFrame_features))
dataFrame_features ['LABEL']= df_copy['Label']
dataFrame_features.head()
dataFrame_features.tail()



In [ ]:
#@title Perfoming correlation to see what features can be used to best train the model
# a correaltion analysis was performed and if a feature had a correlation of more
# than o.5 to the label it was used as a feature to train the model.
df_correlated = dataFrame_features.corr()
#print(df_correlated, '\n')
df_correlated.tail()

In [ ]:
correlation_matrix = dataFrame_features ['numbers_presence'].corr(dataFrame_features['LABEL'])
print(correlation_matrix)


*italicized text*
# we will use:
# total_number_words, numbers_presence, word_but_presence, word_inconclusion_presence,
# repetitive_words and Sentiment

#The correlation threshold used was 0.5/-0.5

In [ ]:
#@title splitting the dataFrame_features into train and test data
train_df = dataFrame_features.sample(frac=0.9, random_state=42)
test_df= dataFrame_features.drop(train_df.index)

# shuffle the training set to avoid overfitting and prevent bias introduced by sequency of the data
train_df = train_df.reindex(np.random.permutation(train_df.index))

train_df.head()
test_df.tail()

In [ ]:
# @title Calulating the z-scores in the train set
# Calculate the z-scores of each column in the training set and
# write those z-scores in a new pandas DataFrame named train_df_norm.
label_column = 'LABEL'

# Select columns for normalization (excluding the label column)
columns_to_normalize = train_df.columns[train_df.columns != label_column]

# Calculate mean and standard deviation for the selected columns
train_df_mean = train_df[columns_to_normalize].mean()
train_df_std = train_df[columns_to_normalize].std()

column1= train_df[['total_number_words', 'numbers_presence', 'word_but_presence', 'word_inconclusion_presence', 'repetitive_words', 'Sentiment']].std()
print (column1)
# Calculate z-scores for the selected columns
train_df_norm = (train_df[columns_to_normalize] - train_df_mean) / train_df_std

# Combine the normalized columns with the non-normalized ones, including the label
train_df_norm_with_label = pd.concat([train_df[label_column], train_df_norm], axis=1)

# Examine some of the values of the normalized training set
print("Standard DEV \n", train_df_std, "\n")
print("Mean \n", train_df_mean, "\n")
train_df_norm_with_label.head()

In [ ]:
# Assuming 'label_column' is the column you want to exclude from normalization
label_column = 'LABEL'

# Select columns for normalization (excluding the label column)
columns_to_normalize_test = test_df.columns[test_df.columns != label_column]

# Calculate z-scores for the selected columns in the test set
test_df_norm = (test_df[columns_to_normalize_test] - train_df_mean[columns_to_normalize_test]) / train_df_std[columns_to_normalize_test]

# Combine the normalized columns with the non-normalized ones, including the label
test_df_norm_with_label = pd.concat([test_df[label_column], test_df_norm], axis=1)

# Examine some of the values of the normalized test set
test_df_norm_with_label.head()

In [ ]:
#@title Represent features as input layer
inputs = {
    # Features used to train the model on.
      'total_number_words': tf.keras.Input(shape=(1,)),
      'numbers_presence': tf.keras.Input(shape=(1,)),
      'word_but_presence': tf.keras.Input(shape=(1,)),
      'word_inconclusion_presence': tf.keras.Input(shape=(1,)),
      'repetitive_words': tf.keras.Input(shape=(1,)),
      'Sentiment': tf.keras.Input(shape=(1,))

}

In [ ]:
#@title  Define functions that build and train the model
def create_model(my_inputs, my_learning_rate, METRICS):
  # Use a Concatenate layer to concatenate the input layers into a single tensor.
  # as input for the Dense layer. Ex: [input_1[0][0], input_2[0][0]]
  concatenated_inputs = tf.keras.layers.Concatenate()(my_inputs.values())

  # activation set to sigmoid for binary classification models
  dense = layers.Dense(units=1, name='dense_layer', activation=tf.sigmoid)
  dense_output = dense(concatenated_inputs)

  """ Create and compile a simple classification model. """
  my_outputs = {
      'dense': dense_output,
  }
  model = tf.keras.Model(inputs=my_inputs, outputs=my_outputs)

  # call the compile method to construct the layers into a model that
  # Tensorflow can execute. Notice that we are using a different loss function
  # for classification than for regression(BinaryCrossentropy())
  # METRIC = is used to asses how well the model is performing on the training
  # and validation data
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=my_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=METRICS)
  return model


def train_model(model, dataset, epochs, label_name,
                batch_size=None, shuffle=True, validation_split=0.1):
  """ Feed a dataset into the model in order to train it."""
  # The x parameter of tf.keras.Model.fit can be a list of arrays, where
  # each array contains the data for one feature . Here, we are passing
  # every column in the dataset. Note that the feature_layer will filter
  # away most of those columns, leaving only the desired colums and their
  # representations as features.
  features = {name:np.array(value) for name, value in dataset.items()}
  label = np.array(features.pop(label_name))
  history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_split=validation_split)


  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch

  #Isolate the classification metric for each epoch.
  hist = pd.DataFrame(history.history)

  return epochs, hist

  print("Defined the create and train_model functions.")



In [ ]:
# @title Define the plotting function
def  plot_curve(epochs, hist, list_of_metrics):
  """plot a curve of one or more classification metrics vs. epoch."""
  #list_of_metrics should be one of the names shown in:
  # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Defined the plot_curve function.")


In [ ]:
#@title Invoke the creating, training and plotting functions
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 25
batch_size = 64
label_name = "LABEL"
classification_threshold = 0.50

# Establish the metrics the model will measure.
METRICS = [
           tf.keras.metrics.BinaryAccuracy(name='accuracy',
                                    threshold=classification_threshold),
           # Add precision and recall as metrics
           tf.keras.metrics.Precision(thresholds=classification_threshold,
                                      name='precision'),

           tf.keras.metrics.Recall(thresholds=classification_threshold,
                                      name='recall'),
           #summarize the model's perfomance
           tf.keras.metrics.AUC(num_thresholds=100, name='auc')

           ]
# Remove precision, recall and auc from the METRICS list
METRICS = [metric for metric in METRICS if 'val_' not in metric.name]

# Establish the models' topography.
my_model = create_model(inputs, learning_rate, METRICS)

# To view a PNG of this model's layers click File>'my_classification_model.png'
# file
tf.keras.utils.plot_model(my_model, "my_classification_model.png")

# Train the model on the training set.
epochs, hist = train_model(my_model, train_df_norm_with_label, epochs,
                           label_name, batch_size)

# Plot a graph of the metric(s) vs. epochs.
list_of_metrics_to_plot = ['accuracy', 'precision', 'recall']

plot_curve(epochs, hist, list_of_metrics_to_plot)
#Accuracy should gradually improve during training
#(untill it can improve no more)

#plotting the auc curve(metrics vs epoch)
list_of_metric_to_plot2 = ['auc']
plot_curve(epochs, hist, list_of_metric_to_plot2)

my_model.save('/content/drive/MyDrive/fantasy.h5')

In [ ]:
#@title Evaluate the model against the test set
# Determine the model's accuracy against the test set.

features = {name:np.array(value) for name, value in test_df_norm_with_label.items()}
label = np.array(features.pop(label_name))

my_model.evaluate(x = features, y = label, batch_size=batch_size)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Sample input text
input_text = "By the end of the 25 epochs, the model has achieved a high accuracy, precision, recall, and AUC, and a low loss on both the training and validation sets. The high recall (1.0000) indicates that the model is correctly identifying all positive instances, while the high precision (0.8780) indicates that the majority of instances classified as positive are indeed positive. The high AUC (0.9627) indicates that the model is performing well in distinguishing between positive and negative instances."
def preprocess_text(text):
    # Tokenize the text
    tokens = re.split(r'\W+', text.lower())
    # Remove stopwords
    tokens = [token for token in tokens if token not in ENGLISH_STOP_WORDS]
    # Rejoin the tokens into a single string
    processed_text = ' '.join(tokens)
    return processed_text

input_text = preprocess_text(input_text)

# Calculate the features for the input text
def count_total_words(text):
    paragraphs = text.split('\n\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    total_word_count = 0

    for paragraph in paragraphs:
        sentences = re.split(r'[.?!+]', paragraph)
        for sentence in sentences:
            words = sentence.split()
            total_word_count += len(words)

    return total_word_count

def detect_total_numbers(text):
    paragraphs = text.split('\n\n')

    has_number = any(any(char.isdigit() for char in paragraph) for paragraph in paragraphs)

    return 1 if has_number else 0

def detect_word_but_total(text):
    paragraphs = text.split('\n\n')
    has_word_but = any('but' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_but else 0

def detect_word_in_conclusion_total(text):
    paragraphs = text.split('\n\n')
    has_word_in_conclusion = any('conclusion' in paragraph.lower() for paragraph in paragraphs)
    return 1 if has_word_in_conclusion else 0

def detect_repetitive_language(text):
    tokens = nltk.word_tokenize(text)
    freq_dist = nltk.FreqDist(tokens)

    # Calculate the frequency of the most common words
    most_common_words = sorted(freq_dist, key=freq_dist.get, reverse=True)[:10]
    num_common_words = sum(freq_dist[word] > 5 for word in most_common_words)

    if num_common_words > 5:
        return 1
    else:
        return 0


from nltk.sentiment import SentimentIntensityAnalyzer

# Download VADER lexicon.
nltk.download('vader_lexicon')


def analyze_sentiment(text):
    # Initialize VADER sentiment analyzer
    sia = SentimentIntensityAnalyzer()

    # Calculate compound sentiment score for the entire text
    sentiment_score = sia.polarity_scores(text)['compound']

    # Classify sentiment based on the compound score
    if sentiment_score >= 0.05:
        sentiment = 1 # for positive
    elif sentiment_score <= -0.05:
        sentiment = -1 # for negative
    else:
        sentiment = 0

    return sentiment

features = {
    'total_number_words': count_total_words(input_text),
    'numbers_presence': detect_total_numbers(input_text),
    'word_but_presence': detect_total_apostrophe(input_text),
    'word_inconclusion_presence': detect_word_in_conclusion_total(input_text),
    'repetitive_words': detect_repetitive_language(input_text),
    'Sentiment': analyze_sentiment(input_text)
}

# Define the names of the features
features_names = ['total_number_words', 'numbers_presence', 'word_but_presence', 'word_inconclusion_presence', 'repetitive_words', 'Sentiment']

# Normalize the feature values using the mean and standard deviation from the training set
train_df_mean = [316.2, 0.4, 0.5, 0.6, 0.7, 0.9]
train_df_std = [77.4, 0.5, 0.5, 0.5, 0.5, 0.5]
features = (np.array(list(features.values())).reshape(1, -1) - train_df_mean) / train_df_std

# Convert the NumPy array back to a dictionary
features_dict = {name: value for name, value in zip(features_names, features[0])}

# Load the saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/fantasy.h5')

# Create a dictionary of input tensors
input_tensors = {name:tf.constant(value.reshape(1,), dtype=tf.float32) for name, value in features_dict.items()}

# Make predictions
predictions = model.predict(input_tensors)

for name, value in predictions.items():
    print(f"{name}: {value.shape}")

# Extract the prediction probablity
probability = predictions['dense'][0][0]
# Define the probability threshold

#threshold = 0.5

# Print the classification result
if probability > classification_threshold:
    print("The text is likely AI-generated.")
else:
    print("The text is likely human-generated.")

print(probability)

In [ ]:
# @title saving a pickle file to use for deployment
import pickle

# Save the model as a pickle file
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

